In [ ]:
pip install ultralytics

In [ ]:
#For Broiler and Laying Hen Detection and labeling
import os
import torch
from clip import load as load_clip  # CLIP
from ultralytics import YOLO
import matplotlib.pyplot as plt
from transformers import CLIPProcessor, CLIPModel, CLIPVisionModelWithProjection, CLIPTextModelWithProjection
from PIL import Image, ImageDraw
import clip

from PIL import Image, ImageDraw


# Load CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = load_clip("ViT-B/32", device=device)

# Load YOLO model
yolo_model = YOLO("/weights/best.pt")  # Replace with your trained model

# Input image directory and text prompts
image_dir = "/images/"  # Directory containing images
text_prompts = {
    "": "",
    "": "",
}

# Function to crop image using bounding box
def crop_image(image, box):
    x1, y1, x2, y2 = map(int, box[:4])  # YOLO box format: x1, y1, x2, y2
    cropped = image.crop((x1, y1, x2, y2))
    return preprocess(cropped).unsqueeze(0).to(device)

# Encode text features for both classes
text_features = {}
for label, prompt in text_prompts.items():
    text_input = clip.tokenize([prompt]).to(device)  # Use the clip.tokenize method to properly tokenize the text
    text_features[label] = clip_model.encode_text(text_input).detach()

# Output directory for results
output_dir = "/Outputs/"
os.makedirs(output_dir, exist_ok=True)

# Process each image in the directory
for file_name in os.listdir(image_dir):
    file_path = os.path.join(image_dir, file_name)
    if not file_name.lower().endswith((".jpg", ".jpeg", ".png")):
        continue  # Skip non-image files

    image = Image.open(file_path).convert("RGB")  # Load and ensure RGB format
    results = yolo_model.predict(source=file_path, conf=0.5, save=False)  # Run YOLO model
    
    # Debugging: Print detection results
    print(f"Processing {file_name}")
    print("Detection results:", results)

    # Filter YOLO detections and process each bounding box
    output_boxes = []
    for detection in results[0].boxes:  # Access the first image's results
        box = detection.xyxy[0].cpu().numpy()  # Bounding box coordinates
        cls_id = int(detection.cls[0])  # Class ID
        class_name = yolo_model.names[cls_id]  # Get class name

        print(f"Detected: {class_name} with confidence {detection.conf[0]}")  # Debugging print
        
        if class_name in text_features:  # Check if the detected class is Broiler or Hen
            cropped_region = crop_image(image, box)  # Crop detected region
            cropped_region_features = clip_model.encode_image(cropped_region).detach()
            
            # Compute similarity for the specific class
            similarity = torch.nn.functional.cosine_similarity(
                text_features[class_name], cropped_region_features
            )
            print(f"Similarity score for {class_name}: {similarity.item()}")  # Debugging print
            if similarity.item() > 0.5:  # Threshold for similarity
                output_boxes.append((box, class_name, similarity.item()))

    # Draw bounding boxes and save results
    draw = ImageDraw.Draw(image)
    for box, label, score in output_boxes:
        x1, y1, x2, y2 = map(int, box)
        draw.rectangle([x1, y1, x2, y2], outline="red", width=3)
        draw.text((x1, y1 - 10), f"{label}: {score:.2f}", fill="red")

    # Save the output image
    output_path = os.path.join(output_dir, f"detection_{file_name}")
    image.save(output_path)
    print(f"Detection results saved to {output_path}")


In [ ]:
#CLIP with YOLO models final model with behavior labels and images output

import os
import torch
from PIL import Image, ImageDraw
import clip
from ultralytics import YOLO

# Load CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device)

# Load YOLO model
yolo_model = YOLO("/weights/best.pt")  # Replace with your trained model

# Input image directory and text prompts
image_dir = "/images/"  # Directory containing images
text_prompts = {
    "": "",
    "": "",
    "": "",
    "": "",
}

# Function to crop image using bounding box
def crop_image(image, box):
    x1, y1, x2, y2 = map(int, box[:4])  # YOLO box format: x1, y1, x2, y2
    cropped = image.crop((x1, y1, x2, y2))
    return preprocess(cropped).unsqueeze(0).to(device)

# Encode text features for both classes
text_features = {}
for label, prompt in text_prompts.items():
    text_input = clip.tokenize([prompt]).to(device)  # Tokenize the prompt
    text_features[label] = clip_model.encode_text(text_input).detach()

# Output directory for results
output_dir = "/Predictedimages/"
os.makedirs(output_dir, exist_ok=True)

# Labels directory for YOLO format labels
labels_dir = "/Predictedlabels/"
os.makedirs(labels_dir, exist_ok=True)

# Process each image in the directory
for file_name in os.listdir(image_dir):
    file_path = os.path.join(image_dir, file_name)
    if not file_name.lower().endswith((".jpg", ".jpeg", ".png")):
        continue  # Skip non-image files

    image = Image.open(file_path).convert("RGB")  # Load and ensure RGB format
    results = yolo_model.predict(source=file_path, conf=0.5, save=True)  # Lower confidence threshold
    
    print(f"Processing {file_name}")
    print("Detection results:", results)  # Debugging print
    
    # Check if results are empty
    if not results[0].boxes:
        print(f"No detections for {file_name}")
        continue  # Skip image if no detections
    
    # Filter YOLO detections and process each bounding box
    output_boxes = []
    for detection in results[0].boxes:  # Access the first image's results
        box = detection.xyxy[0].cpu().numpy()  # Bounding box coordinates
        cls_id = int(detection.cls[0])  # Class ID
        class_name = yolo_model.names[cls_id]  # Get class name

        print(f"Detected: {class_name} with confidence {detection.conf[0]}")  # Debugging print
        
        # Check if the detected class name matches any of the text prompts
        matching_label = None
        for label, prompt in text_prompts.items():
            if prompt.lower() in class_name.lower():  # Case-insensitive match
                matching_label = label
                break

        if matching_label:  # If a match is found for the label
            cropped_region = crop_image(image, box)  # Crop detected region
            cropped_region_features = clip_model.encode_image(cropped_region).detach()
            
            # Compute similarity for the specific class
            similarity = torch.nn.functional.cosine_similarity(
                text_features[matching_label], cropped_region_features
            )
            print(f"Similarity score for {matching_label}: {similarity.item()}")  # Debugging print
            if similarity.item() > 0.2:  # Lower the similarity threshold
                output_boxes.append((box, matching_label, similarity.item()))

    # Draw bounding boxes and save results
    draw = ImageDraw.Draw(image)
    yolo_labels = []  # To store YOLO formatted labels

    for box, label, score in output_boxes:
        x1, y1, x2, y2 = map(int, box)
        draw.rectangle([x1, y1, x2, y2], outline="blue", width=3)
        draw.text((x1, y1 - 10), f"{label}: {score:.2f}", fill="blue")

        # Convert to YOLO format: <class_id> <x_center> <y_center> <width> <height>
        x_center = (x1 + x2) / 2 / image.width
        y_center = (y1 + y2) / 2 / image.height
        width = (x2 - x1) / image.width
        height = (y2 - y1) / image.height

        # Get the class ID from the class name
        cls_id = list(text_prompts.keys()).index(label)  # Get the class ID from the text_prompts

        # Append to YOLO label list
        yolo_labels.append(f"{cls_id} {x_center} {y_center} {width} {height}")

    # Show the image for debugging
    image.show()

    # Save the output image
    output_path = os.path.join(output_dir, f"detection_{file_name}")
    image.save(output_path)
    print(f"Detection results saved to {output_path}")

    # Save YOLO format labels to a text file
    label_path = os.path.join(labels_dir, f"{os.path.splitext(file_name)[0]}.txt")
    with open(label_path, 'w') as label_file:
        for label in yolo_labels:
            label_file.write(label + "\n")
    print(f"YOLO labels saved to {label_path}")
